In [1]:
from pandas_datareader import data as pdr
from yahoo_fin import stock_info as si
import yfinance as yf
import pandas as pd
import datetime
import time
import talib
from talib import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from xgboost import XGBClassifier
import xgboost as xgb
import math
import numpy as np
from sklearn.decomposition import PCA
import pickle
import os
os.environ["PATH"] += os.pathsep + r'C:\Program Files\Graphviz\bin'
yf.pdr_override()

In [2]:
from LSSVMRegression import LSSVMRegression

In [3]:
def add_indicators():
        
    R_S_I = RSI(df['Adj Close'], timeperiod=slow)
    E_M_A = EMA(df['Adj Close'], timeperiod=fast)
    macd, macdsig, macdhist = MACD(df['Adj Close'], fastperiod=fast, slowperiod=slow, signalperiod=really_fast)
    mfi = MFI(df['High'], df['Low'], df['Adj Close'],df['Volume'],timeperiod=fast)
    slowk, slowd = STOCH(df['High'], df['Low'], df['Adj Close'], fastk_period=fast, slowk_period=slow, slowk_matype=0, slowd_period=slow, slowd_matype=0)
    
    #
    #
    #
    
    df['ema'] = E_M_A
    df['rsi'] = R_S_I
    df['macd'] = macd
    df['macdsig'] = macdsig
    df['macdhist'] = macdhist
    df['mfi'] = mfi
    df['slowK'] = slowk
    df['slowD'] = slowd
    
    
    
    ####
    
    

    #76 vars

    #are_all_zero = (test_TA == 0).all()
    #true if all values are 0
    #false if contain a non 0'''

    df.drop(['Close'], axis =1, inplace = True)

    

In [4]:
def add_lag(num_lag_cols, this_df):
    
    tic = time.perf_counter()
    
    '''df['ema'] = E_M_A
    df['rsi'] = R_S_I
    df['macd'] = macd
    df['macdsig'] = macdsig
    df['macdhist'] = macdhist
    df['mfi'] = mfi
    df['slowK'] = slowk
    df['slowD'] = slowd'''
    
    lag_cols = ['open',
                 'high',
                 'low',
                 'adj_close',
                 'volume',
                 'ema',
                 'rsi',
                 'macd',
                 'macdsig',
                 'macdhist',
                 'mfi',
                 'slowk',
                 'slowd']

    shift_range = [x+1 for x in range(num_lag_cols)]

    for shift in shift_range:
        train_shift = this_df[merging_keys + lag_cols].copy()

        # E.g. order_day of 0 becomes 1, for shift = 1.
        # So when this is merged with order_day of 1 in df, this will represent lag of 1.
        train_shift['order_day'] = train_shift['order_day'] + shift

        foo = lambda x: '{}_lag_{}'.format(x, shift) if x in lag_cols else x
        train_shift = train_shift.rename(columns=foo)

        this_df = pd.merge(this_df, train_shift, on=merging_keys, how='left') #.fillna(0)

    del train_shift
    
    toc = time.perf_counter()

    tic_toc = (toc - tic) / 60

    print(f"completed lagging in {tic_toc:0.4f} min")
    

    return this_df

In [5]:
def add_scale(num_interval_lag):

    cols_to_scale = ['open',#
                     'high',#
                     'low',#
                     'adj_close',#
                     'volume',#
                     'ema',#
                     'rsi',#
                     'macd',#
                     'macdsig',#
                     'macdhist',#
                     'mfi',#
                     'slowk',#
                     'slowd']#
                    

    for i in range(1,num_interval_lag+1):
        cols_to_scale.append("ema_lag_"+str(i))
        cols_to_scale.append("slowk_lag_"+str(i))
        cols_to_scale.append("slowd_lag_"+str(i))
        cols_to_scale.append("rsi_lag_"+str(i))

        cols_to_scale.append("volume_lag_"+str(i))

        cols_to_scale.append("adj_close_lag_"+str(i))
        
        cols_to_scale.append("open_lag_"+str(i))
        cols_to_scale.append("high_lag_"+str(i))
        cols_to_scale.append("low_lag_"+str(i))

        cols_to_scale.append("macd_lag_"+str(i))
        cols_to_scale.append("macdsig_lag_"+str(i))
        cols_to_scale.append("macdhist_lag_"+str(i))

        cols_to_scale.append("mfi_lag_"+str(i))
      
    return cols_to_scale

In [6]:
stock = 'AAPL'

start_date = datetime.datetime.now() - datetime.timedelta(days=59)

end_date = datetime.datetime.now()

df = pdr.get_data_yahoo(stock, start=start_date, end=end_date, interval = "2m", prepost = True)

#df = pdr.get_data_yahoo(stock, period = "max", interval = "1d", prepost = True)

#df.index = df.index.tz_localize(None)

'''#2 min ticker
# 30 intervals = 1 hour << OLD
# 195 intervals = trading day'''
                                            # there are more intervals that we can use / change
really_fast = 30
fast = 60
slow = 90

add_indicators()

# Convert Date column to datetime
df.reset_index(level=0, inplace=True)

# Change all column headings to be lower case, and remove spacing
df.columns = [str(x).lower().replace(' ', '_') for x in df.columns]

df['order_day'] = [x for x in list(range(len(df)))]
# merging_keys


# Get difference between high and low of each day

[*********************100%***********************]  1 of 1 completed


In [7]:
col_list = df.columns.to_list()
col_list.remove('datetime')

In [8]:
col_list

['open',
 'high',
 'low',
 'adj_close',
 'volume',
 'ema',
 'rsi',
 'macd',
 'macdsig',
 'macdhist',
 'mfi',
 'slowk',
 'slowd',
 'order_day']

In [9]:
merging_keys = ['order_day']

num_interval_lag = 30

df = add_lag(num_interval_lag, df)

#df['adj_close'] = df['adj_close'].shift(30)

completed lagging in 0.0338 min


In [10]:
print(df.shape)

(15206, 405)


In [11]:
print(df.head(40))

                    datetime    open    high     low  adj_close  volume  ema  \
0  2021-01-13 19:00:00-05:00  131.00  131.03  130.95     131.03       0  NaN   
1  2021-01-13 19:02:00-05:00  131.00  131.03  131.00     131.03       0  NaN   
2  2021-01-13 19:04:00-05:00  131.03  131.07  131.00     131.00       0  NaN   
3  2021-01-13 19:06:00-05:00  131.00  131.07  131.00     131.05       0  NaN   
4  2021-01-13 19:08:00-05:00  131.05  131.06  131.00     131.02       0  NaN   
5  2021-01-13 19:10:00-05:00  131.06  131.06  131.01     131.02       0  NaN   
6  2021-01-13 19:12:00-05:00  131.02  131.06  131.01     131.05       0  NaN   
7  2021-01-13 19:14:00-05:00  131.02  131.07  131.01     131.07       0  NaN   
8  2021-01-13 19:16:00-05:00  131.06  131.08  131.05     131.08       0  NaN   
9  2021-01-13 19:18:00-05:00  131.08  131.10  131.05     131.10       0  NaN   
10 2021-01-13 19:20:00-05:00  131.10  131.10  131.05     131.07       0  NaN   
11 2021-01-13 19:22:00-05:00  131.08  13

In [12]:
window_size = 30

df['adj_close'] = df['adj_close'].shift(-window_size)



In [13]:
cols_to_rem = ['open',
                 'high',
                 'low',
                 'volume',
                 'ema',
                 'rsi',
                 'macd',
                 'macdsig',
                 'macdhist',
                 'mfi',
                 'slowk',
                 'slowd',
                 'order_day']

In [14]:
close_vals_list = df['adj_close']

In [15]:
df.drop(cols_to_rem, axis = 1, inplace = True)

In [16]:
print(df.tail(40))

                       datetime  adj_close  open_lag_1  high_lag_1  low_lag_1  \
15166 2021-03-02 17:44:00-05:00     125.32     125.030      125.08    125.020   
15167 2021-03-02 17:46:00-05:00     125.33     125.030      125.07    125.030   
15168 2021-03-02 17:48:00-05:00     125.35     125.050      125.08    125.010   
15169 2021-03-02 17:50:00-05:00     125.33     125.045      125.07    125.020   
15170 2021-03-02 17:52:00-05:00     125.34     125.070      125.08    125.050   
15171 2021-03-02 17:54:00-05:00     125.34     125.080      125.08    125.020   
15172 2021-03-02 17:56:00-05:00     125.29     125.070      125.08    125.010   
15173 2021-03-02 17:58:00-05:00     125.30     125.015      125.02    125.000   
15174 2021-03-02 18:00:00-05:00     125.24     125.010      125.05    125.000   
15175 2021-03-02 18:02:00-05:00     125.27     125.020      125.20    125.000   
15176 2021-03-02 18:04:00-05:00        NaN     125.190      125.23    125.130   
15177 2021-03-02 18:06:00-05

In [17]:


#close_vals_list = df['adj_close']

df.fillna(0, inplace=True)

#df.fillna(0, inplace=True)

#BUY => 1, SELL => 0, HOLD => 2

num_test = int(.10*len(df))
num_train = len(df) - num_test

# Split into train, cv, and test
train = df[:num_train]
test = df[num_train:]

#print(test['datetime'].iloc[[-1]])
#print(test['test_labels'].iloc[[-1]])

train_close_vals = close_vals_list[:num_train]
test_close_vals = close_vals_list[num_train:]

#test_dates_list = test['date']
#BUY => 1, SELL => 0, HOLD => 2

In [18]:
test_dates_list = test['datetime']

In [19]:
train_dates_list = train['datetime']

In [20]:
test_adj_close_list = close_vals_list[num_train:]

In [21]:
cols_to_scale = add_scale(num_interval_lag)
#cols_to_scale.remove(cols_to_rem)

# Do scaling for train set
# Here we only scale the train dataset, and not the entire dataset to prevent information leak


In [22]:
for element in cols_to_rem:

    if element in cols_to_scale:

        cols_to_scale.remove(element)

In [30]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import normalize

In [24]:
scaler = StandardScaler()

scaler.fit(train[cols_to_scale])
train_scaled = scaler.transform(train[cols_to_scale])

# Convert the numpy array back into pandas dataframe

train_scaled = pd.DataFrame(train_scaled, columns=cols_to_scale)

In [36]:


# Convert the numpy array back into pandas dataframe

train_scaled_3, norms = normalize(train[cols_to_scale], return_norm=True)

In [37]:
train_scaled_3 = pd.DataFrame(train_scaled_3, columns=cols_to_scale)


In [25]:
scaler_2 = Normalizer()

scaler_2.fit(train[cols_to_scale])
train_scaled_2 = scaler_2.transform(train[cols_to_scale])

# Convert the numpy array back into pandas dataframe

train_scaled_2 = pd.DataFrame(train_scaled_2, columns=cols_to_scale)

In [28]:
train_scaled['open_lag_1']

0       -25.156084
1        -0.490500
2        -0.490500
3        -0.484852
4        -0.490500
           ...    
13681    -2.316883
13682    -2.322533
13683    -2.373369
13684    -2.330063
13685    -2.335712
Name: open_lag_1, Length: 13686, dtype: float64

In [29]:
train_scaled_2['open_lag_1']

0        0.000000
1        0.447207
2        0.333314
3        0.277378
4        0.242506
           ...   
13681    0.000023
13682    0.000020
13683    0.000013
13684    0.000013
13685    0.000013
Name: open_lag_1, Length: 13686, dtype: float64

In [38]:
train_scaled_3['open_lag_1']

0        0.000000
1        0.447207
2        0.333314
3        0.277378
4        0.242506
           ...   
13681    0.000023
13682    0.000020
13683    0.000013
13684    0.000013
13685    0.000013
Name: open_lag_1, Length: 13686, dtype: float64

In [43]:
de_normed = train_scaled_3['open_lag_1'] * norms

In [44]:
de_normed

0          0.000000
1        131.000000
2        131.000000
3        131.030000
4        131.000000
            ...    
13681    121.300003
13682    121.269997
13683    121.000000
13684    121.230000
13685    121.200000
Name: open_lag_1, Length: 13686, dtype: float64

In [45]:
train['open_lag_1']

0          0.000000
1        131.000000
2        131.000000
3        131.030000
4        131.000000
            ...    
13681    121.300003
13682    121.269997
13683    121.000000
13684    121.230000
13685    121.200000
Name: open_lag_1, Length: 13686, dtype: float64

In [ ]:
#scaler = StandardScaler()

scaler = StandardScaler()

scaler.fit(train[cols_to_scale])
train_scaled = scaler.transform(train[cols_to_scale])

# Convert the numpy array back into pandas dataframe

train_scaled = pd.DataFrame(train_scaled, columns=cols_to_scale)

train_scaled = train_scaled[slow:]
train = train[slow:]

#scaling test dataset

#scaler_2 = StandardScaler()

#scaler_2  = MinMaxScaler(feature_range=(-1,1))
#scaler.fit(test[cols_to_scale])
test_scaled = scaler.transform(test[cols_to_scale])

# Convert the numpy array back into pandas dataframe

test_scaled = pd.DataFrame(test_scaled, columns=cols_to_scale)

features = cols_to_scale

In [ ]:
features

In [ ]:
features.remove('adj_close')

In [ ]:
print('adj_close' in features)

In [ ]:
print('adj_close' in train_scaled.columns)

In [ ]:
#features.remove(target)

# Split into X and y
'''X_train_scaled = train_scaled[features]
y_train_scaled = train['labels']

X_test_scaled = test_scaled[features]
y_test_scaled = test['labels']'''

X_train = train_scaled[features]
y_train = train_scaled['adj_close']

X_test = test_scaled[features]
y_test = test_scaled['adj_close']

In [ ]:
import pyswarms as ps

from sklearn.metrics import mean_squared_error

In [ ]:
def fitness_function(position):
    
    #print(position)
    
    #svrRegressor = SVR(kernel='rbf', gamma = position[0].all(), C = position[1].all() )
    
    clfrbf=LSSVMRegression(
        gamma=position[0].all(),       #set the gamma-hyper parameter equal to 1
        kernel='rbf', #use the linear kernel
        sigma=position[1].all(),
            )
    
    clfrbf.fit(X_train, y_train)
    
    y_train_pred = clfrbf.predict(X_train)
    
    y_test_pred = clfrbf.predict(X_test)
    
    mse_test =mean_squared_error(y_test, y_test_pred)
    
    rmse_test = math.sqrt(mse_test)
    
    print('Optimizing the Parameters ..... C = {c}, Gamma={gamma}'.format(c=position[1], gamma=position[0]))
    
    mse_train = mean_squared_error(y_train, y_train_pred)
    
    rmse_train = math.sqrt(mse_train)
    
    #list(range(mse_f_train,rmse_f_train))
    retVal= rmse_test #Return a vector instead of single value. you can use the rmse_test and ignore rmse_train
    
    return retVal

In [ ]:
lb = [10, 0.0001]
ub = [1000,0.1]
#mb = [100, 0.001]
bounds = (lb,ub)

options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
optimizer = ps.single.GlobalBestPSO(n_particles=20, dimensions=2,options=options, bounds = bounds)
gamma_opt, C_opt = optimizer.optimize(fitness_function, iters=50)

In [ ]:
print(gamma_opt)
print(C_opt)

In [ ]:
clfrbf=LSSVMRegression(
        gamma=C_opt[0],       #set the gamma-hyper parameter equal to 1
        kernel='rbf', #use the linear kernel
        sigma=C_opt[1]
            )

In [ ]:
clfrbf.fit(X_train, y_train)

In [ ]:
y_hat = clfrbf.predict(X_test)

In [ ]:
y_hat_train = clfrbf.predict(X_train)

In [ ]:
test_scaled['adj_close'] = y_hat

inv_scaler = scaler.inverse_transform(test_scaled)

y_hat_descaled = pd.DataFrame(inv_scaler, columns = test_scaled.columns)

In [ ]:
these_params = clfrbf.get_params()

In [ ]:
print(these_params)

In [ ]:
import pickle
# this methodology works for saving a trained model
pickle.dump(clfrbf, open("test.model", "wb"))

In [ ]:
import joblib as joblib
from joblib import dump
filename = "test.joblib"
joblib.dump(clfrbf, filename)

In [ ]:
import dill

with open("test", "wb") as dill_file:
    dill.dump(clfrbf, dill_file)

In [ ]:
this_save = dill.dumps(clfrbf) 

In [ ]:
this_load = pd.read_pickle(r'test.model')

In [ ]:
y_hat = this_load.predict(X_test)

In [ ]:
with open("test.model", "wb") as dill_file:
    dill.dump(this_save, dill_file)

In [ ]:
this_open = dill.load("test.model") 

In [ ]:
#with open("test.model", "rb") as dill_file:
    test_clf = dill.load(this_open)

In [ ]:
y_hat_descaled['adj_close']

In [ ]:
%matplotlib notebook
plt.rcParams['figure.figsize'] = [10, 6]

plt.plot(test_dates_list, y_hat_descaled['adj_close'])
plt.plot(test_dates_list, test_close_vals)
plt.legend(['pred','true'])
plt.show

In [ ]:
y_hat = clfrbf.predict(X_train)

In [ ]:
train_scaled['adj_close'] = y_hat

inv_scaler = scaler.inverse_transform(train_scaled)

y_hat_descaled = pd.DataFrame(inv_scaler, columns = train_scaled.columns)

In [ ]:
%matplotlib notebook
plt.rcParams['figure.figsize'] = [10, 6]

plt.plot(train_dates_list[slow:], y_hat_descaled['adj_close'])
plt.plot(train_dates_list[slow:], train_close_vals[slow:])
plt.legend(['pred','true'])
plt.show